In [1]:
import os
from torch.utils.data import DataLoader
from trajdata import AgentBatch, UnifiedDataset

In [2]:
dataset = UnifiedDataset(
    desired_data=["eupeds_eth","eupeds_hotel","eupeds_univ","eupeds_zara1","eupeds_zara2" ],
    data_dirs={  # Remember to change this to match your filesystem!
        "eupeds_eth": "../data/eth",
        "eupeds_hotel": "../data/eth",
        "eupeds_univ": "../data/eth",
        "eupeds_zara1": "../data/eth",
        "eupeds_zara2": "../data/eth",
    },
)

In [3]:
dataloader = DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=dataset.get_collate_fn(),
    num_workers=os.cpu_count(), # This can be set to 0 for single-threaded loading, if desired.
)

In [4]:
data_iter = iter(dataloader)
batch = next(data_iter) 